In [4]:
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        print("Found a GPU with the name:", gpu)
else:
    print("Failed to detect a GPU. Using CPU.")

# Example: Create a simple operation and specify device
with tf.device('/GPU:0' if gpus else '/CPU:0'):
    a = tf.constant([[1.0, 2.0], [4.0, 5.0]])
    print("Tensor created on:", a.device)

Failed to detect a GPU. Using CPU.
Tensor created on: /job:localhost/replica:0/task:0/device:CPU:0


In [3]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [1]:
import os
import shutil

def sort_videos(source_folder):
    """
    Separates video files into 'real' and 'fake' directories based on naming convention.

    Args:
        source_folder (str): The path to the folder containing all video files.
    """

    # Define the output directories
    real_videos_dir = os.path.join(source_folder, 'real_videos')
    fake_videos_dir = os.path.join(source_folder, 'fake_videos')

    # Create the output directories if they don't exist
    os.makedirs(real_videos_dir, exist_ok=True)
    os.makedirs(fake_videos_dir, exist_ok=True)

    print(f"Sorting videos from: {source_folder}")
    print(f"Real videos will go to: {real_videos_dir}")
    print(f"Fake videos will go to: {fake_videos_dir}\n")

    # List all files in the source folder
    try:
        all_files = os.listdir(source_folder)
    except FileNotFoundError:
        print(f"Error: Source folder '{source_folder}' not found. Please check the path.")
        return

    # Counter for sorted files
    real_count = 0
    fake_count = 0
    unclassified_count = 0

    for filename in all_files:
        file_path = os.path.join(source_folder, filename)

        # Skip directories
        if os.path.isdir(file_path):
            continue

        # Get file name without extension and its extension
        name_without_ext, ext = os.path.splitext(filename)
        ext = ext.lower() # Convert extension to lowercase for consistent checking

        # Check if it's a common video file extension
        # You can add more extensions here if needed
        if ext not in ['.mp4', '.avi', '.mkv', '.mov', '.wmv', '.flv']:
            print(f"Skipping non-video file: {filename}")
            unclassified_count += 1
            continue

        # Determine if it's a real or fake video based on the underscore
        if '_' in name_without_ext:
            # It's a fake video (e.g., 000_003.mp4)
            destination_path = os.path.join(fake_videos_dir, filename)
            try:
                shutil.move(file_path, destination_path)
                print(f"Moved '{filename}' to 'fake_videos'")
                fake_count += 1
            except Exception as e:
                print(f"Error moving '{filename}' to 'fake_videos': {e}")
        else:
            # It's a real video (e.g., 000.mp4)
            destination_path = os.path.join(real_videos_dir, filename)
            try:
                shutil.move(file_path, destination_path)
                print(f"Moved '{filename}' to 'real_videos'")
                real_count += 1
            except Exception as e:
                print(f"Error moving '{filename}' to 'real_videos': {e}")

    print(f"\n--- Sorting Complete ---")
    print(f"Total 'real' videos moved: {real_count}")
    print(f"Total 'fake' videos moved: {fake_count}")
    print(f"Total unclassified/skipped files: {unclassified_count}")

# --- How to use the script ---
if __name__ == "__main__":
    # IMPORTANT: Replace 'path/to/your/video/folder' with the actual path
    # where your videos (e.g., 000.mp4, 001_870.mp4) are located.
    # Example: 'C:/Users/YourUser/Videos/MyDatasets' or '/home/youruser/videos/datasets'
    
    # !!! DANGER !!!
    # Ensure the 'source_video_folder' is correct.
    # This script will move files, not copy them.
    # It's recommended to back up your data before running.
    source_video_folder = 'E:/deepfake/deepfake' 
    
    sort_videos(source_video_folder)

Sorting videos from: E:/deepfake/deepfake
Real videos will go to: E:/deepfake/deepfake\real_videos
Fake videos will go to: E:/deepfake/deepfake\fake_videos

Skipping non-video file: .gitattributes
Moved '000.mp4' to 'real_videos'
Moved '000_003.mp4' to 'fake_videos'
Moved '001.mp4' to 'real_videos'
Moved '001_870.mp4' to 'fake_videos'
Moved '002.mp4' to 'real_videos'
Moved '002_006.mp4' to 'fake_videos'
Moved '003.mp4' to 'real_videos'
Moved '003_000.mp4' to 'fake_videos'
Moved '004.mp4' to 'real_videos'
Moved '004_982.mp4' to 'fake_videos'
Moved '005.mp4' to 'real_videos'
Moved '005_010.mp4' to 'fake_videos'
Moved '006.mp4' to 'real_videos'
Moved '006_002.mp4' to 'fake_videos'
Moved '007.mp4' to 'real_videos'
Moved '007_132.mp4' to 'fake_videos'
Moved '008.mp4' to 'real_videos'
Moved '008_990.mp4' to 'fake_videos'
Moved '009.mp4' to 'real_videos'
Moved '009_027.mp4' to 'fake_videos'
Moved '010.mp4' to 'real_videos'
Moved '010_005.mp4' to 'fake_videos'
Moved '011.mp4' to 'real_videos'


In [3]:
import os
import shutil
import re
from collections import defaultdict

def organize_videos(
    source_dir,
    selected_videos_dir,
    remaining_videos_dir,
    num_to_select_per_id_pair=10,
    video_extensions=('.mp4', '.mov', '.avi', '.mkv', '.webm')
):
    print("--- Starting Video Organization ---")
    print(f"Source Directory: {source_dir}")
    print(f"Selected Videos Destination: {selected_videos_dir}")
    print(f"Remaining Videos Destination: {remaining_videos_dir}")
    print(f"Number of videos to select per ID pair: {num_to_select_per_id_pair}")
    print(f"Video extensions to process: {', '.join(video_extensions)}")
    print("-" * 40)

    try:
        os.makedirs(selected_videos_dir, exist_ok=True)
        os.makedirs(remaining_videos_dir, exist_ok=True)
        print("Destination directories ensured.")
    except OSError as e:
        print(f"Error creating directories: {e}")
        print("Please check directory paths and permissions. Exiting.")
        return

    # Regex to extract 'idX_idY' from filenames like 'id0_id1_0000.mp4'
    # This pattern will match id followed by digits, underscore, id followed by digits,
    # then underscore and more digits, followed by a dot and any extension.
    id_pair_pattern = re.compile(r'^(id\d+)_(id\d+)_\d+\..+')

    selected_count = 0
    remaining_count = 0
    
    # This defaultdict tracks how many videos have been selected for each normalized ID pair.
    # 'id0_id1' and 'id1_id0' will be treated as the same, e.g., always 'id0_id1'.
    processed_id_pairs_count = defaultdict(int)

    for filename in os.listdir(source_dir):
        file_path = os.path.join(source_dir, filename)

        if os.path.isfile(file_path) and filename.lower().endswith(video_extensions):
            destination_path = None
            
            match = id_pair_pattern.match(filename)
            
            if match:
                id1, id2 = match.groups()
                # Normalize the ID pair by sorting for consistent grouping (e.g., 'id0_id1' for both 'id0_id1' and 'id1_id0')
                normalized_id_pair = '_'.join(sorted([id1, id2]))

                if processed_id_pairs_count[normalized_id_pair] < num_to_select_per_id_pair:
                    destination_path = os.path.join(selected_videos_dir, filename)
                    try:
                        shutil.move(file_path, destination_path)
                        selected_count += 1
                        processed_id_pairs_count[normalized_id_pair] += 1
                    except shutil.Error as e:
                        print(f"Error moving '{filename}' to selected directory: {e}")
                        print(f"File might already exist or path is invalid. Skipping.")
                else:
                    # If enough videos for this ID pair are already selected, move to 'remaining'
                    destination_path = os.path.join(remaining_videos_dir, filename)
                    try:
                        shutil.move(file_path, destination_path)
                        remaining_count += 1
                    except shutil.Error as e:
                        print(f"Error moving '{filename}' to remaining directory: {e}")
                        print(f"File might already exist or path is invalid. Skipping.")
            else:
                # If filename does not match the 'idX_idY_numbers' pattern, move to 'remaining'
                destination_path = os.path.join(remaining_videos_dir, filename)
                try:
                    shutil.move(file_path, destination_path)
                    remaining_count += 1
                except shutil.Error as e:
                    print(f"Error moving '{filename}' to remaining directory: {e}")
                    print(f"File might already exist or path is invalid. Skipping.")

    print("-" * 40)
    print("Video organization complete!")
    print(f"Successfully moved {selected_count} videos to '{selected_videos_dir}'.")
    print(f"Successfully moved {remaining_count} videos to '{remaining_videos_dir}'.")
    print(f"Total video files processed and moved: {selected_count + remaining_count}")
    print("--- End of Script ---")

# --- How to use the script ---
# IMPORTANT: Uncomment the block below and set your actual paths before running the script.

# Line 106: Set the path to your original video dataset folder.
source_directory = 'E:/deepfake/media/datasets/fake'

# Line 109: Set the path where selected videos will be moved.
selected_dest_directory = 'E:/deepfake/media/datasets/fake'

# Line 112: Set the path where remaining videos will be moved.
remaining_dest_directory = 'E:/deepfake/media/datasets'

# Line 115: This determines how many videos to select per ID pair (e.g., 10).
videos_per_id = 10 

# Uncomment the line below to run the function after setting paths:
organize_videos(source_directory, selected_dest_directory, remaining_dest_directory, videos_per_id)


--- Starting Video Organization ---
Source Directory: E:/deepfake/media/datasets/fake
Selected Videos Destination: E:/deepfake/media/datasets/fake
Remaining Videos Destination: E:/deepfake/media/datasets
Number of videos to select per ID pair: 10
Video extensions to process: .mp4, .mov, .avi, .mkv, .webm
----------------------------------------
Destination directories ensured.
----------------------------------------
Video organization complete!
Successfully moved 3364 videos to 'E:/deepfake/media/datasets/fake'.
Successfully moved 0 videos to 'E:/deepfake/media/datasets'.
Total video files processed and moved: 3364
--- End of Script ---


In [6]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))

if gpus:
    print("GPU details:")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True) # Important for better memory management
        print(gpu)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(logical_gpus), "Logical GPUs")
else:
    print("No GPU detected by TensorFlow. Please check your CUDA/cuDNN installation and environment variables.")

# Optional: Run a simple operation to confirm it uses the GPU
try:
    with tf.device('/GPU:0'):
        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
        c = tf.matmul(a, b)
        print("\nSimple matrix multiplication on GPU:")
        print(c)
except RuntimeError as e:
    print("\nCould not run on GPU:", e)

TensorFlow version: 2.19.0
Num GPUs Available:  0
No GPU detected by TensorFlow. Please check your CUDA/cuDNN installation and environment variables.

Simple matrix multiplication on GPU:
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [4]:
import csv
import os
import shutil

def split_videos(metadata_filepath, videos_directory=None):
    """
    Reads a metadata CSV file to categorize video filenames as 'REAL' or 'FAKE'.
    If a videos_directory is provided, it moves files into 'real' and 'fake' subdirectories.

    Args:
        metadata_filepath (str): The path to the metadata CSV file.
        videos_directory (str, optional): The base directory where video files are located.
                                          If None, the script will only print the lists.
    """
    real_videos = []
    fake_videos = []

    try:
        with open(metadata_filepath, mode='r', newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            # Assuming the CSV has two columns: video_filename, label
            for row in reader:
                if len(row) == 2:
                    video_filename = row[0].strip()
                    label = row[1].strip().upper() # Convert to uppercase for consistent comparison
                    
                    if label == 'REAL':
                        real_videos.append(video_filename)
                    elif label == 'FAKE':
                        fake_videos.append(video_filename)
                    else:
                        print(f"Warning: Unknown label '{row[1]}' for video '{row[0]}'. Skipping.")
                else:
                    print(f"Warning: Skipping malformed row: {row}")

    except FileNotFoundError:
        print(f"Error: Metadata file not found at '{metadata_filepath}'")
        return
    except Exception as e:
        print(f"An error occurred while reading the CSV: {e}")
        return

    print("--- Real Videos ---")
    if real_videos:
        for video in real_videos:
            print(video)
    else:
        print("No real videos found.")

    print("\n--- Fake Videos ---")
    if fake_videos:
        for video in fake_videos:
            print(video)
    else:
        print("No fake videos found.")

    # Move files if a directory is provided
    if videos_directory:
        print(f"\n--- Moving files in '{videos_directory}' ---")
        real_dir = os.path.join(videos_directory, 'real_videos')
        fake_dir = os.path.join(videos_directory, 'fake_videos')

        # Create target directories if they don't exist
        try:
            os.makedirs(real_dir, exist_ok=True)
            os.makedirs(fake_dir, exist_ok=True)
            print(f"Directories created (or already exist): '{real_dir}' and '{fake_dir}'")
        except OSError as e:
            print(f"Error creating directories: {e}")
            return

        # Move real videos
        for video in real_videos:
            source_path = os.path.join(videos_directory, video)
            destination_path = os.path.join(real_dir, video)
            if os.path.exists(source_path):
                try:
                    shutil.move(source_path, destination_path)
                    print(f"Moved '{video}' to '{real_dir}'")
                except Exception as e:
                    print(f"Error moving '{video}' to '{real_dir}': {e}")
            else:
                print(f"Warning: Real video file '{video}' not found in '{videos_directory}'. Skipping move.")

        # Move fake videos
        for video in fake_videos:
            source_path = os.path.join(videos_directory, video)
            destination_path = os.path.join(fake_dir, video)
            if os.path.exists(source_path):
                try:
                    shutil.move(source_path, destination_path)
                    print(f"Moved '{video}' to '{fake_dir}'")
                except Exception as e:
                    print(f"Error moving '{video}' to '{fake_dir}': {e}")
            else:
                print(f"Warning: Fake video file '{video}' not found in '{videos_directory}'. Skipping move.")
        print("\nFile splitting complete.")
    else:
        print("\nNo 'videos_directory' provided. Files were not moved.")


# Example usage:
# Assuming metadata.csv is in the same directory as the script.
# Replace 'E:/deepfake/media/FF_Face_only_data' with the actual path where your .mp4 files are stored.
split_videos('E:/deepfake/media/FF_Face_only_data/metadata.csv', videos_directory='E:/deepfake/media/FF_Face_only_data')


--- Real Videos ---
000.mp4
001.mp4
002.mp4
003.mp4
004.mp4
005.mp4
006.mp4
007.mp4
008.mp4
009.mp4
010.mp4
011.mp4
012.mp4
013.mp4
014.mp4
015.mp4
016.mp4
017.mp4
018.mp4
019.mp4
020.mp4
021.mp4
022.mp4
023.mp4
024.mp4
025.mp4
026.mp4
027.mp4
028.mp4
029.mp4
030.mp4
031.mp4
032.mp4
033.mp4
034.mp4
035.mp4
036.mp4
037.mp4
038.mp4
039.mp4
040.mp4
041.mp4
042.mp4
043.mp4
044.mp4
045.mp4
046.mp4
047.mp4
048.mp4
049.mp4
050.mp4
051.mp4
052.mp4
053.mp4
054.mp4
055.mp4
056.mp4
057.mp4
058.mp4
059.mp4
060.mp4
061.mp4
062.mp4
063.mp4
064.mp4
065.mp4
066.mp4
067.mp4
068.mp4
069.mp4
070.mp4
071.mp4
072.mp4
073.mp4
074.mp4
075.mp4
076.mp4
077.mp4
078.mp4
079.mp4
080.mp4
081.mp4
082.mp4
083.mp4
084.mp4
085.mp4
086.mp4
087.mp4
088.mp4
089.mp4
090.mp4
091.mp4
092.mp4
093.mp4
094.mp4
095.mp4
096.mp4
097.mp4
098.mp4
099.mp4
100.mp4
101.mp4
102.mp4
103.mp4
104.mp4
105.mp4
106.mp4
107.mp4
108.mp4
109.mp4
110.mp4
111.mp4
112.mp4
113.mp4
114.mp4
115.mp4
116.mp4
117.mp4
118.mp4
119.mp4
120.mp4
121.mp4
122.